In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.font_manager
from collections import OrderedDict
import csv

In [2]:
features = ['sens', 'cont']

def spawn_heatmap(benchmarks = []):
    bb = ['bodytrack', 'blackscholes', 'canneal', 'dealII', 'moses', 'sphinx3', 'milc', 'astar', 'tailbench.sphinx', 'ferret', 'leslie3d', 'silo', 'omnetpp', 'zeusmp', 'gobmk', 'povray', 'x264', 'calculix', 'bwaves', 'facesim', 'gromacs', 'perlbench', 'shore', 'bzip2', 'hmmer', 'namd', 'h264ref', 'fluidanimate', 'swaptions', 'cactusADM', 'GemsFDTD', 'freqmine', 'masstree', 'img-dnn', 'mcf', 'dedup', 'streamcluster', 'vips', 'gamess', 'xalancbmk', 'sjeng', 'soplex', 'gcc', 'tonto', 'lbm', 'libquantum']
    vcpus = ['1', '2', '4', '8']
    excluded_benchmarks = ['shore-1', 'masstree-8']
    if benchmarks == []:
        benchmarks = [x + '-' + y for x in bb for y in vcpus]
    benchmarks = list(filter(lambda x: x not in excluded_benchmarks, benchmarks))
    return OrderedDict((b, OrderedDict()) for b in benchmarks)

In [3]:
def clean_heatmap(heatmap):
    for b in list(filter(lambda x: heatmap[x] == {}, list(heatmap.keys()))): del heatmap[b]

In [4]:
def read_heatmap(heatmap, name = 'heatmap'):
    hm_file = name + '.csv'
    try:
        fd = open(hm_file)
    except:
        write_heatmap()
        fd = open(hm_file)
    hm_read = csv.reader(fd, delimiter='\t')
    new_files = []
    for row in hm_read:
        if row[0] == '':
            benchmarks = row
        else:
            bench1 = row[0]
            if bench1 not in heatmap.keys(): continue
            for (i, sd) in enumerate(row):
                if i == 0: continue
                bench2 = benchmarks[i]
                if bench2 not in heatmap.keys(): continue
                if float(sd) > 0:
                    heatmap[bench1][bench2] = float(sd)
    fd.close()
    clean_heatmap(heatmap)


In [5]:
def box_plot_classvsclass(qos, class_):
    def classes_from_file(qos, class_):
        classes = dict()
        for feature in features:
            classes_fd = open(f"SVC_{feature}_{class_}_{qos}_spt_cv.csv")
            classes[feature] = dict((rows[0], int(rows[-1]))
                                    for rows in filter(lambda x: x[0] != 'Bench' and x[0] != 'Accuracy',
                                                csv.reader(classes_fd, delimiter='\t')))
            classes_fd.close()
        return classes

    heatmap = spawn_heatmap()
    read_heatmap(heatmap)
    classes = classes_from_file(qos, class_)
    per_classes = [[{'sd': []} for c in set(classes['cont'].values())] for s in set(classes['sens'].values())]

    for bench1 in heatmap:
        for bench2 in heatmap[bench1]:
            per_classes[classes['sens'][bench1]][classes['cont'][bench2]]['sd'].append(heatmap[bench1][bench2])
    dataframes = []
    for s in per_classes:
        for c in s:
            dataframes.append(pd.DataFrame(c).assign(Class = str(per_classes.index(s)) + 'vs' + str(s.index(c))))

    df = pd.concat(dataframes)
    df = df.pivot(columns = 'Class', values = 'sd')
    bp = df.boxplot(grid = False, figsize = (20, int(class_ == 3) * 2.5 + 7.5), patch_artist = True,
                    rot = (0 if class_ == 2 else 90), return_type = 'both')
    labels = {2: ['Insensitive -\nNon-Contentious', 'Insensitive -\nContentious', \
                  'Sensitive -\nNon-Contentious', 'Sensitive -\nContentious'],
              3: ['Insensitive -\nNon-Cont.', 'Insensitive -\nMod. Cont.', 'Insensitive -\nContentious',
                  'Mod. Sens. -\nNon-Cont.', 'Mod. Sens. -\nMod. Cont.', 'Mod. Sens. -\nContentious',
                  'Sensitive -\nNon-Cont.', 'Sensitive -\nMod. Cont.', 'Sensitive -\nContentious']}
    color1 = 'dimgray'
    color2 = 'silver'
    lw = 3
    fontsize = 45
    fname = "Times New Roman"
    (y_bottom, y_top, y_interval) = (1.0, min(2.4,float(class_)), qos - 1.0)

    bp[0].set_xticklabels(labels[class_])
    bp[0].set_yticks(np.arange(y_bottom, y_top, y_interval))

    for whisker in bp[1]['whiskers']: whisker.set(color = color1, linewidth = lw)
    for cap in bp[1]['caps']: cap.set(color = color1, xdata = cap.get_xdata() - (-0.005, +0.005), linewidth = lw)
    for box in bp[1]['boxes']:
        box.set(color = color1, linewidth = lw)
        box.set_facecolor(color2)
    for median in bp[1]['medians']: median.set(color=color1, linewidth = lw)
    for flier in bp[1]['fliers']: flier.set(marker = 'o', markerfacecolor = color2, markeredgecolor = color1, markersize = 6)
    for tick in bp[0].xaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize)
        tick.label1.set_fontname(fname)
    for tick in bp[0].yaxis.get_major_ticks():
        tick.label1.set_fontsize(fontsize - 6)
        tick.label1.set_fontname(fname)

    plt.ylabel('Slowdown', fontsize = fontsize + 6, fontname = fname)
    plt.ylim(bottom = y_bottom, top = y_top)
    plt.axhline(y = qos, color = 'firebrick', linewidth = lw + 1, zorder = 4)
    plt.suptitle('')
    plt.tight_layout()
    plt.savefig(f"ClassVClass_{class_}_{qos}.png")
    plt.clf()


In [6]:
for c in [2, 3]:
    box_plot_classvsclass(1.2, c)

<Figure size 2000x750 with 0 Axes>